# Spark SQL

Spark SQL essentially tries to bridge the gap between the two models we mentioned previously—the relational and procedural models.

Spark SQL provides a DataFrame API that can perform relational operations on both external data sources and Spark's built-in distributed collections—at scale!

To support a wide variety of diverse data sources and algorithms in Big Data, Spark SQL introduces a novel extensible optimizer called Catalyst, which makes it easy to add data sources, optimization rules, and data types for advanced analytics such as machine learning. Essentially, Spark SQL leverages the power of Spark to perform distributed, robust, in-memory computations at massive scale on Big Data.

## Read the csv data 

In [33]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Basics').getOrCreate()
df = spark.read.csv('../data/auction.csv',header=True, inferSchema=True)
df

DataFrame[auctionid: bigint, bid: double, bidtime: double, bidder: string, bidderrate: string, openbid: double, price: double, item: string, auction_type: string]

## Show()

In [11]:
df.show(2)

+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
| auctionid|  bid| bidtime|      bidder|bidderrate|openbid|price|              item| auction_type|
+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
|1638893549|175.0|2.230949|schadenfreud|         0|   99.0|177.5|Cartier wristwatch|3 day auction|
|1638893549|100.0|2.600116|       chuik|         0|   99.0|177.5|Cartier wristwatch|3 day auction|
+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
only showing top 2 rows



## Data Schema

In [13]:
df.printSchema()

root
 |-- auctionid: long (nullable = true)
 |-- bid: double (nullable = true)
 |-- bidtime: double (nullable = true)
 |-- bidder: string (nullable = true)
 |-- bidderrate: string (nullable = true)
 |-- openbid: double (nullable = true)
 |-- price: double (nullable = true)
 |-- item: string (nullable = true)
 |-- auction_type: string (nullable = true)



## Get the columns

In [15]:
df.columns

['auctionid',
 'bid',
 'bidtime',
 'bidder',
 'bidderrate',
 'openbid',
 'price',
 'item',
 'auction_type']

## Describe()

In [21]:
df.describe().select("summary","price").show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|             10681|
|   mean|335.04358861528874|
| stddev| 433.5660087308641|
|    min|              26.0|
|    max|            5400.0|
+-------+------------------+



## Summary()

In [24]:
df.summary().select("summary","price").show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|             10681|
|   mean|335.04358861528874|
| stddev| 433.5660087308641|
|    min|              26.0|
|    25%|            186.51|
|    50%|            228.01|
|    75%|             255.0|
|    max|            5400.0|
+-------+------------------+



## Custom Data Schema

In [31]:
from pyspark.sql.types import StructField, IntegerType, StringType, StructType
address_schema = [StructField('city',StringType(),True),StructField('state',StringType(),True)]
final_add_schema = StructType(fields=address_schema)
data_schema = [StructField('id',IntegerType(),True),StructField('name',StringType(),True),
               StructField('address',final_add_schema,True)]
final_struc = StructType(fields=data_schema)
df1 = spark.read.json('../data/data.json',schema=final_struc)
df1.show()

+---+----+----------------+
| id|name|         address|
+---+----+----------------+
|  1|Lijo|[Columbus, Ohio]|
+---+----+----------------+



## Head()

In [37]:
df.head(2)
dict=df.head(2)[0].asDict()
dict

{'auctionid': 1638893549,
 'bid': 175.0,
 'bidtime': 2.230949,
 'bidder': 'schadenfreud',
 'bidderrate': '0',
 'openbid': 99.0,
 'price': 177.5,
 'item': 'Cartier wristwatch',
 'auction_type': '3 day auction'}

## Filtering Operations

Filtering can be done with SQL-like syntax or Pythonic way. 

In [47]:
df.filter("price>100").show(2)
# we can also use dataframe to filter
df.filter(df['price'] > 150).show(2)
# use == for equals comparison
df.filter(df['price'] == '177.5').collect()[:2]

+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
| auctionid|  bid| bidtime|      bidder|bidderrate|openbid|price|              item| auction_type|
+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
|1638893549|175.0|2.230949|schadenfreud|         0|   99.0|177.5|Cartier wristwatch|3 day auction|
|1638893549|100.0|2.600116|       chuik|         0|   99.0|177.5|Cartier wristwatch|3 day auction|
+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
only showing top 2 rows

+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
| auctionid|  bid| bidtime|      bidder|bidderrate|openbid|price|              item| auction_type|
+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
|1638893549|175.0|2.230949|schadenfreud|         0|   99.0|177.5|Cartier wristwatch|

[Row(auctionid=1638893549, bid=175.0, bidtime=2.230949, bidder='schadenfreud', bidderrate='0', openbid=99.0, price=177.5, item='Cartier wristwatch', auction_type='3 day auction'),
 Row(auctionid=1638893549, bid=100.0, bidtime=2.600116, bidder='chuik', bidderrate='0', openbid=99.0, price=177.5, item='Cartier wristwatch', auction_type='3 day auction')]

## Create new columns (after some processing of existing columns)

In [51]:
df.withColumn('double_price',df['price']*2).select("price","double_price").show(2)

+-----+------------+
|price|double_price|
+-----+------------+
|177.5|       355.0|
|177.5|       355.0|
+-----+------------+
only showing top 2 rows



## groupBy()

In [59]:
data = df.groupBy("item")
data.max().show(2)
data.count().show(2)

+-------------------+--------------+--------+------------+------------+----------+
|               item|max(auctionid)|max(bid)|max(bidtime)|max(openbid)|max(price)|
+-------------------+--------------+--------+------------+------------+----------+
|Palm Pilot M515 PDA|    3410601613|   290.0|     6.99999|       265.0|     290.0|
| Cartier wristwatch|    1650986455|  5400.0| 6.999965278|      5000.0|    5400.0|
+-------------------+--------------+--------+------------+------------+----------+
only showing top 2 rows

+-------------------+-----+
|               item|count|
+-------------------+-----+
|Palm Pilot M515 PDA| 5917|
| Cartier wristwatch| 1953|
+-------------------+-----+
only showing top 2 rows



## agg()

In [63]:
group_data = df.groupBy('item')
group_data.agg({'price':'max'}).show(2)

+-------------------+----------+
|               item|max(price)|
+-------------------+----------+
|Palm Pilot M515 PDA|     290.0|
| Cartier wristwatch|    5400.0|
+-------------------+----------+
only showing top 2 rows



## orderBy()

In [86]:
df.groupBy('item').max('price').orderBy('max(price)',ascending=False).show()
df.groupBy('item').max('price').sort('max(price)',ascending=True).show()

+-------------------+----------+
|               item|max(price)|
+-------------------+----------+
| Cartier wristwatch|    5400.0|
|  Xbox game console|    501.77|
|Palm Pilot M515 PDA|     290.0|
+-------------------+----------+

+-------------------+----------+
|               item|max(price)|
+-------------------+----------+
|Palm Pilot M515 PDA|     290.0|
|  Xbox game console|    501.77|
| Cartier wristwatch|    5400.0|
+-------------------+----------+



## Using SQL statements

In [93]:
df.createOrReplaceTempView('auctions')
data = spark.sql("select * from auctions")
data.show(2)

+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
| auctionid|  bid| bidtime|      bidder|bidderrate|openbid|price|              item| auction_type|
+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
|1638893549|175.0|2.230949|schadenfreud|         0|   99.0|177.5|Cartier wristwatch|3 day auction|
|1638893549|100.0|2.600116|       chuik|         0|   99.0|177.5|Cartier wristwatch|3 day auction|
+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
only showing top 2 rows



In [95]:
df_sales = spark.sql("SELECT * FROM csv.`../data/vgsales.csv`")
df_sales.show(2)

+----+----------+--------+----+------+---------+--------+--------+--------+-----------+------------+
| _c0|       _c1|     _c2| _c3|   _c4|      _c5|     _c6|     _c7|     _c8|        _c9|        _c10|
+----+----------+--------+----+------+---------+--------+--------+--------+-----------+------------+
|Rank|      Name|Platform|Year| Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|
|   1|Wii Sports|     Wii|2006|Sports| Nintendo|   41.49|   29.02|    3.77|       8.46|       82.74|
+----+----------+--------+----+------+---------+--------+--------+--------+-----------+------------+
only showing top 2 rows



In [102]:
spark.stop()